In [ ]:
import brightway2 as bw
import numpy as np
from pathlib import Path
import pickle, json
from pypardiso import spsolve

#Local files
from import_databases import *
from utils_exiobase import exiobaseLCA

write_dir = Path("write_files")

# 1. Get exiobase scores

In [ ]:
bw.projects.set_current("exiobase_industry_workaround")
exiobase_name = "EXIOBASE 2.2"
ex = bw.Database(exiobase_name)
methods = [
    ('IPCC 2013', 'climate change', 'GTP 100a'),
]

In [ ]:
%%time
# takes 20 min or so
filepath = write_dir / "exiobase_lca.pickle"
if not filepath.exists():
    exiobase_scores_industry_workaround = {}
    for i,act in enumerate(ex):
        if i%500==0:
            print("lca for act {0:4d}/{1}".format(i,len(ex)))
        db_code = (act.as_dict()["database"], act.as_dict()["code"])
        demand = {act: 1}
        lca = bw.LCA(demand, methods[0])
        lca.lci()
        lca.lcia()
        exiobase_scores_industry_workaround[db_code] = {}
        for method in methods:
            lca.switch_method(method)
            lca.redo_lcia()
            exiobase_scores_industry_workaround[db_code].update({
                method: lca.score
            })
    with open(filepath, 'wb') as f:
        pickle.dump(exiobase_scores_industry_workaround, f)
else:
    with open(filepath, 'rb') as f:
        exiobase_scores_industry_workaround = pickle.load(f)

# 2. Import databases

In [ ]:
which_pc = "sasha"

project = "rebound"
bw.projects.set_current(project)
# del bw.databases['CH consumption 1.0']

In [ ]:
if which_pc == 'sasha':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'

In [ ]:
%%time 
# This takes a while
ei36_name = 'ecoinvent 3.6 cutoff'
co_name = CONSUMPTION_DB_NAME

if project == 'rebound':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)

# 3. Compute total score for a given demand

In [ ]:
co = bw.Database(CONSUMPTION_DB_NAME)
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}
sectors = [act for act in co if 'sector' in act["name"]]

In [ ]:
%%time
# This takes 30s or so
exio_lca = exiobaseLCA(
    project, 
    demand,
    exiobase_scores_industry_workaround,
)
scores = exio_lca.compute_total_scores()
scores

# Contribution analysis

In [ ]:
%%time
project = 'rebound'
co = bw.Database("CH consumption 1.0")
contribution_scores = {}
i = 0
# for act in sectors:
for act in list(co):
    demand = {act: 1}
    exio_lca = exiobaseLCA(
        project, 
        demand,
        exiobase_scores_industry_workaround,
    )
    act_name = act["name"]
    score = exio_lca.compute_total_scores()
    contribution_scores[act] = score
    print(i, score[methods[0]], act)
    i+=1

In [ ]:
filepath = write_dir / "contribution_scores.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(contribution_scores, f)

In [ ]:
contribution_scores